In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('customer_coordinates.csv')

In [3]:
customers = df[['Lat', 'Lon']].values.tolist()
demand = df['Demand'].tolist()
N=len(customers)

In [4]:
#Calculate Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in kilometers
    m = 0.0174532925 #degree in radioms
    dlat = (lat2 - lat1)*m
    dlon = (lon2 - lon1)*m
    a = np.sin(dlat / 2) ** 2 + np.cos(lat2*m) * np.cos(lat1*m) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    return distance

In [5]:
# Package costs per unit of product for each warehouse
def C_pk(d):
    if d <= 500:
        return 0.05
    elif 500 < d <= 1000:
        return 0.04
    else:
        return 0.03

Ckm = 1.97 #Operation cost of the vehicle per km
Cop = 20 #Cost of daily operation

In [6]:
def decode_M(x1):
    if x1 <= 1/3:
        return 1
    elif x1 <= 2/3:
        return 2
    else:
        return 3

def decode_hj(x, M):
    h = []
    for j in range(7, len(x)):
        val = x[j]
        if M == 1:
            h.append(1)
        else:
            if val <= 1/M:
                h.append(1)
            elif val <= 2/M:
                h.append(2)
            else:
                h.append(3)
    return h

latmin = 39.6003201
longmin = 20.7660648
latmax = 39.7506861
longmax = 20.9345622
Δlat = latmax - latmin
Δlong = longmax - longmin

def decode_coordinates(x):
    M = decode_M(x[0])
    w = []
    for i in range(M):
        lat = x[i*2+1] * Δlat + latmin
        long = x[i*2+2] * Δlong + longmin
        w.append((lat, long))
    return w

In [7]:
def is_inside_triangle(point, triangle):
    x, y = point
    x1, y1 = triangle[0]
    x2, y2 = triangle[1]
    x3, y3 = triangle[2]
    
    # Calculate the barycentric coordinates
    denom = (y2 - y3) * (x1 - x3) + (x3 - x2) * (y1 - y3)
    alpha = ((y2 - y3) * (x - x3) + (x3 - x2) * (y - y3)) / denom
    beta = ((y3 - y1) * (x - x3) + (x1 - x3) * (y - y3)) / denom
    gamma = 1 - alpha - beta
    
    # Check if the point is inside the triangle
    return 0 <= alpha <= 1 and 0 <= beta <= 1 and 0 <= gamma <= 1

In [8]:
def cost_function(x):
    M = decode_M(x[0])
    w = decode_coordinates(x)
    h = decode_hj(x, M)
    
    total_cost = 0
    # Define the coordinates of the lake triangle
    lake_coords = [(39.688367, 20.838671), (39.632539, 20.901448), (39.666708, 20.929109)]
          
    # Calculate total tij
    t = np.zeros((M, N))
    for i in range(M):
        for j in range(N):
            t[i][j] = haversine(w[i][0], w[i][1], customers[j][0], customers[j][1])
    
    # Calculate total cost Ci
    total_cost = 0
    for i in range(M):
        sdi = sum([demand[j] for j in range(N)])
        sti = sum([t[i][j] for j in range(N)])
        Ci = abs(Cop + sdi * C_pk(sdi)) + sti * Ckm
        
        
        if is_inside_triangle(w[i], lake_coords):
            # Add penalty to the total cost
            total_cost += 2000 + Ci
        else:
            total_cost += Ci
        
    return total_cost

In [15]:
def initialization(X, population_size):
    Pk = []
    for _ in range(population_size):
        Pk.append(np.random.uniform(size=len(X)))
    return Pk
#Fitness διαμέσου σύγκρισης με τον χειρότερο του πληθυσμού
def fitness_function(x, worst_cost):
    cost = cost_function(x)
    fitness = worst_cost - cost
    return fitness
#roulette-wheel selection
def selection(Pk, fitness):
    total_fitness = np.sum(fitness)
    probabilities = fitness / total_fitness
    selected_indices = np.random.choice(len(Pk), size=len(Pk), p=probabilities)
    Sk = [Pk[i] for i in selected_indices]
    return Sk

#real-valued recombination
def crossover(Sk, delta=0.15):
    
    # Initialize the offspring
    Ck = []
    
    # Perform crossover for each pair of individuals
    for i in range(0, len(Sk), 2):
        parent1 = Sk[i]
        parent2 = Sk[i+1]
        
        # Perform real-valued recombination
        child1 = np.zeros(len(parent1))
        child2 = np.zeros(len(parent2))
        for j in range(len(parent1)):
            Rj = np.random.uniform(-delta, 1 + delta)
            child1[j] = Rj * parent1[j] + (1 - Rj) * parent2[j]
            child2[j] = Rj * parent2[j] + (1 - Rj) * parent1[j]
        
        # Add the offspring to the new population
        Ck.append(child1)
        Ck.append(child2)
    
    return Ck

#real-valued mutation
def mutation(Ck, lower_bound=0, upper_bound=1):
    # Initialize the mutated population
    Mk = []
    
    # Perform mutation for each individual
    for individual in Ck:
        # Perform mutation operation (e.g., random mutation)
        mutated_individual = individual.copy()
        for j in range(len(mutated_individual)):
            if np.random.random() < 0.6:
                # Calculate the closest edge of the search space
                closest_edge = lower_bound if abs(lower_bound - mutated_individual[j]) < abs(upper_bound - mutated_individual[j]) else upper_bound
                # Calculate standard deviation based on the distance to the closest edge
                sigma = abs(closest_edge - mutated_individual[j]) / 3 
                # Generate a random number with normal distribution
                zj = np.random.normal(0, sigma**2)
                # Check if the new value is within bounds
                mutated_individual[j] += zj       
        
        # Add the mutated individual to the new population
        Mk.append(mutated_individual)
    return Mk

def new_population(Mk, Pk):
    # Combine the parent and mutated populations
    combined_population = Pk + Mk
    # Sort the combined population based on fitness
    sorted_population = sorted(combined_population, key=lambda x: cost_function(x))
    # Select the top population_size individuals
    new_population = sorted_population[:population_size]
       
    return new_population

def update_best(Pk, fitness):
    best_index = np.argmin(fitness)
    xb = Pk[best_index]
    return xb

In [11]:
#GENETIC ALGORITHM
def genetic_algorithm(X, cost_function, population_size):
    computation_budget = 200000
    computation_count = 0

    Pk = initialization(X, population_size)
    costs = [cost_function(x) for x in Pk]
    worst_cost = max(costs)
    fitness = [fitness_function(x, worst_cost) for x in Pk]
    
    computation_count += len(Pk)

    xb = update_best(Pk, fitness)
    best_cost = cost_function(xb)
    best_budget = computation_count

    while computation_count <computation_budget:
        
        # Selection
        Sk = selection(Pk, fitness)
        computation_count += 1

        # Crossover
        Ck = crossover(Sk)
        computation_count += 1

        # Mutation
        Mk = mutation(Ck)
        computation_count += 1

        # Evaluate the new population
        costs = [cost_function(x) for x in Mk]
        computation_count += len(Mk)

        worst_cost = max(costs)
        new_fitness = [fitness_function(x, worst_cost) for x in Mk]
        computation_count += len(Mk)

        # Create the new population
        Pk = new_population(Mk, Pk)
        computation_count += 1

        # Update the best solution
        xb = update_best(Pk, new_fitness)
        computation_count += 1

        new_cost = cost_function(xb)
        if new_cost < best_cost:
            best_cost = new_cost
            best_budget = computation_count
            
    print(best_cost, (best_budget/200000)*100)
    return xb

In [12]:
X = np.array([0.599] + [0.823, 0.695, 0.517, 0.550, 0.034, 0.439] + [0.674, 0.456, 0.789, 0.123, 0.567, 0.890, 0.345, 0.678, 0.901, 0.234, 0.789, 0.111, 0.456, 0.789, 0.333, 0.666, 0.222, 0.888, 0.444, 0.777, 0.555, 0.999, 0.111, 0.25, 0.48, 0.6, 0.3, 0.75, 0.95, 0.15, 0.85, 0.55, 0.28, 0.62, 0.37, 0.98, 0.18, 0.42, 0.72, 0.57, 0.82, 0.23, 0.67, 0.39, 0.93, 0.19, 0.47, 0.77, 0.53, 0.87])
population_size = 200

In [13]:
# Initialize DataFrame for results with an index
results_df = pd.DataFrame(columns=['Result'])

# Repeat the genetic algorithm and append results to the DataFrame
for run_number in range(1, 26):  # Adjust the number of runs as needed
    X_random = np.random.random(size=X.shape)
    result = genetic_algorithm(X_random, cost_function, population_size)
    results_df.loc[run_number] = [' '.join(map(str, result))]

# Print the DataFrame with results
print(results_df)

# Save results to a file
results_df.to_csv('wheel200.txt', index=True)

448.44957946819875 71.58250000000001
448.4495794681987 87.58
448.4495794681987 51.535
448.44957946819875 82.9225
448.44957946819875 87.985
448.44957946819875 56.8
448.44957946819875 69.76
448.44957946819875 64.29249999999999
448.44957946819875 85.555
448.44957946819875 89.2
448.44957946819875 95.0725
448.44957946819875 53.7625
448.44957946819875 40.6
448.44957946819875 84.1375
448.44957946819875 53.559999999999995
448.44957946819875 70.975
448.4495794681988 44.4475
448.44957946819875 51.129999999999995
448.44957946819875 67.1275
448.44957946819875 79.07499999999999
448.44957946819875 60.2425
448.44957946819875 85.555
448.44957946819875 54.572500000000005
448.4495794681987 48.699999999999996
448.44957946819875 46.0675
                                               Result
1   0.21219859481067094 0.44773373408131206 0.4384...
2   0.18776483576466127 0.44773373363689994 0.4384...
3   0.19225952516233974 0.4477337368379245 0.43847...
4   0.2018657280660287 0.44773373445185594 0.43847...
5  

In [ ]:
#GENETIC WITH TOURNAMENT SELECTION

In [10]:
def tournament_selection(Pk, fitness, tournament_size):
    selected_parents = []
    
    for _ in range(len(Pk)):
        tournament_indices = np.random.choice(len(Pk), size=tournament_size, replace=False)
        tournament_fitness = [fitness[i] for i in tournament_indices]
        winner_index = tournament_indices[np.argmax(tournament_fitness)]
        selected_parents.append(Pk[winner_index])
    
    return selected_parents

def genetic_algorithm(X, cost_function, population_size, tournament_size):
    computation_budget = 200000
    computation_count = 0
    
    Pk = initialization(X, population_size)
    costs = [cost_function(x) for x in Pk]
    worst_cost = max(costs)
    fitness = [fitness_function(x, worst_cost) for x in Pk]
    
    computation_count += len(Pk)
    
    xb = update_best(Pk, fitness)
    best_cost = cost_function(xb)
    best_budget = computation_count
    
    while computation_count <computation_budget:
        # Tournament selection
        Sk = tournament_selection(Pk, fitness, tournament_size)
        computation_count += 1
        # Crossover
        Ck = crossover(Sk)
        computation_count += 1
        # Mutation
        Mk = mutation(Ck)
        computation_count += 1
        # Evaluate the new population
        costs = [cost_function(x) for x in Mk]
        computation_count += len(Mk)
        
        worst_cost = max(costs)
        new_fitness = [fitness_function(x, worst_cost) for x in Mk]
        computation_count += len(Mk)
        
        # Create the new population
        Pk = new_population(Mk, Pk)
        computation_count += 1
        # Update the best solution
        xb = update_best(Pk, new_fitness)
        computation_count += 1
        
        new_cost = cost_function(xb)
        if new_cost < best_cost:
            best_cost = new_cost
            best_budget = computation_count
    
    print(best_cost, (best_budget/200000)*100)
    return xb

In [11]:
X = np.array([0.599] + [0.823, 0.695, 0.517, 0.550, 0.034, 0.439] + [0.674, 0.456, 0.789, 0.123, 0.567, 0.890, 0.345, 0.678, 0.901, 0.234, 0.789, 0.111, 0.456, 0.789, 0.333, 0.666, 0.222, 0.888, 0.444, 0.777, 0.555, 0.999, 0.111, 0.25, 0.48, 0.6, 0.3, 0.75, 0.95, 0.15, 0.85, 0.55, 0.28, 0.62, 0.37, 0.98, 0.18, 0.42, 0.72, 0.57, 0.82, 0.23, 0.67, 0.39, 0.93, 0.19, 0.47, 0.77, 0.53, 0.87])

# Create a random array with the same shape as X
X_random = np.random.random(size=X.shape)
tournament_size =30
population_size=200

# Initialize DataFrame for results with an index
results_df = pd.DataFrame(columns=['Result'])

# Repeat the genetic algorithm and append results to the DataFrame
for run_number in range(1, 26):  # Adjust the number of runs as needed
    X_random = np.random.random(size=X.shape)
    result = genetic_algorithm(X_random, cost_function, population_size, tournament_size)
    results_df.loc[run_number] = [' '.join(map(str, result))]

# Print the DataFrame with results
print(results_df)

# Save results to a file
results_df.to_csv('tourn200.txt', index=True)

448.4495794681988 26.424999999999997
448.4495794681988 48.902499999999996
448.44957946819875 50.5225
448.4495794681988 65.71000000000001
448.44957946819875 70.975
448.4495794681988 26.627499999999998
448.4495794681988 26.2225
448.4495794681987 62.875
448.44957946819875 97.9075
448.44957946819875 86.36500000000001
448.4495794681988 33.917500000000004
448.44957946819875 64.495
448.4495794681988 25.412499999999998
448.44957946819875 77.86
448.44957946819875 43.03
448.44957946819875 74.8225
448.44957946819875 45.6625
448.44957946819875 67.9375
448.44957946819875 99.9325
448.44957946819875 60.6475
448.44957946819875 88.795
448.44957946819875 63.685
448.44957946819875 48.902499999999996
448.44957946819875 87.58
448.4495794681988 50.5225
                                               Result
1   0.14848550902251711 0.4477337348735979 0.43847...
2   0.21994089312320558 0.44773373548177675 0.4384...
3   0.16116871130753435 0.4477337364949286 0.43847...
4   0.1116716339399707 0.44773373243160863 

In [ ]:
#PSO

In [35]:
#LBEST

In [9]:
def initialization(X):
    # Initialize particles randomly within the search space (X)
    return np.random.uniform(low=X[0], high=X[1], size=(num_particles, dimension))

def initialization_velocity(S):
    # Initialize velocities to be zero
    return np.zeros_like(S)

def get_ring_lbest(P, i, ρ):
    # Get the indices of the ρ nearest neighbors in a ring topology
    neighbors = [(i + offset) % len(P) for offset in range(-ρ, ρ + 1)]
    # Apply the cost function to each neighbor and find the index of the one with the minimum cost
    costs = [cost_function(P[j]) for j in neighbors]
    lbest_index = neighbors[np.argmin(costs)]
    # Find the best position among the neighbors
    lbest = P[lbest_index]
    return lbest

def update_velocity(V, S, P, chi, c1, c2, ρ):
    # Update particles' velocities
    for i in range(len(S)):
        lbest = get_ring_lbest(P, i, ρ)
        r1, r2 = np.random.rand(2, *S.shape)  # Cognitive and social random sequences
        V[i] = chi * (V[i] + c1 * r1[i] * (P[i] - S[i]) + c2 * r2[i] * (lbest - S[i]))
    return V
def limit_velocity(V, l, u, alpha):
    vmax = alpha * (u - l)
    return np.clip(V, -vmax, vmax)

def check_bounds(S,X):
    lower_bound_mask = S < X[0]
    upper_bound_mask = S > X[1]
    S = np.clip(S, X[0], X[1])  # Ensure particles' positions are within the search space
    return S
def update_particles(V, S):
    # Update particles' positions based on their velocities
    return S + V

def evaluate(S, cost_function):
    fitness = []
    for particle in S:
        fitness.append(cost_function(particle))
    return fitness

def update_best_positions(S, P, fitness, new_fitness):
    # Find where the new fitness is better than the old fitness
    improvement = new_fitness < fitness
    # Update the best positions where an improvement was found
    P[improvement] = S[improvement]
    return P

In [10]:
def pso(cost_function, X, chi, c1, c2, alpha, max_iter):
    # Initialize particles and velocities
    t = 0
    S = initialization(X)
    V = initialization_velocity(S)

    # Initialize the best fitness and position
    fitness = evaluate(S, cost_function)
    P = S.copy()
    global_best_cost = np.inf
    global_best_position = None
    
    while t < max_iter:
        # Update velocities and limit their magnitude
        V = update_velocity(V, S, P, chi, c1, c2, ρ)
        V = limit_velocity(V, X[0], X[1], alpha)

        # Update particles and ensure they are within bounds
        S = update_particles(V, S)
        S = check_bounds(S, X)

        # Update the best fitness and position
        new_fitness = evaluate(S, cost_function)
        # Update best positions
        P = update_best_positions(S, P, fitness, new_fitness)
        fitness = new_fitness
        # Increment time step
        t += num_particles
        
        # Check if current best is better than global best
        current_best_cost = np.min(fitness)
        if current_best_cost < global_best_cost:
            global_best_cost = current_best_cost
            global_best_position = P[np.argmin(fitness)]
            best_budget = t
            
        if t % 20000 == 0:
            S = initialization(X)
            V = initialization_velocity(S)
            P = S.copy()
            
       
    # Report best position
    print(global_best_cost, (best_budget/200000)*100 )
    return global_best_position

In [11]:
# Define the search space
X = [0, 1]  
chi=0.729
c1 = 2.05
c2 = 2.05
ρ=1
# Define the number of particles and the dimension of the problem
num_particles = 200 #adjust the number of particles
dimension =57
alpha=0.01
max_iter=200000

In [12]:
# Initialize DataFrame for results with an index
results_df = pd.DataFrame(columns=['Result'])

# Repeat the genetic algorithm and append results to the DataFrame
for run_number in range(1, 26):  # Adjust the number of runs as needed
    result =  pso(cost_function, X, chi, c1, c2, alpha, max_iter)
    results_df.loc[run_number] = [' '.join(map(str, result))]

# Print the DataFrame with results
print(results_df)

# Save results to a file
results_df.to_csv('lbest200.txt', index=True)

448.4495794735857 99.9
448.44957947896097 29.5
448.4495795075994 90.0
448.4495794692625 9.2
448.4495794765113 79.2
448.44957946821694 79.3
448.4495794964961 79.60000000000001
448.44957950615526 99.1
448.44957947908347 29.9
448.4495794781951 100.0
448.4495794825862 88.7
448.44957948083425 88.7
448.44957947124044 60.0
448.4495794689426 79.80000000000001
448.4495794686102 39.7
448.449579480192 99.9
448.44957947781796 29.9
448.4495794949522 69.8
448.4495798012727 49.1
448.4495794696198 79.80000000000001
448.4495795189636 79.9
448.44957947076756 99.3
448.4495794734005 58.8
448.4495794904226 89.4
448.44957949399634 79.3
                                               Result
1   0.20440244129111987 0.44773076720744687 0.4386...
2   0.30792483099846085 0.44754742668094394 0.4382...
3   0.246223539140396 0.44773544707417273 0.438476...
4   0.2570825591254558 0.4477313769869085 0.438479...
5   0.21754049948434948 0.44775028333646055 0.4384...
6   0.24042411327873617 0.4477343142641257 0.43847...


In [ ]:
#GBEST

In [11]:
def update_velocity_gbest(V, S, P, chi, c1, c2):
    # Find the global best position
    gbest = P[np.argmin([cost_function(p) for p in P])]

    for i in range(len(V)):
        for j in range(len(V[0])):
            R1 = np.random.uniform(0, 1)
            R2 = np.random.uniform(0, 1)
            # Update velocity based on global best position
            V[i][j] = chi * (V[i][j] + c1 * R1 * (P[i][j] - S[i][j]) + c2 * R2 * (gbest[j] - S[i][j]))
    return V

In [12]:
def pso_gbest(cost_function, X, chi, c1, c2, alpha, max_iter):
    # Initialize particles and velocities
    t = 0
    S = initialization(X)
    V = initialization_velocity(S)

    # Initialize the best fitness and position
    fitness = evaluate(S, cost_function)
    P = S.copy()
    global_best_cost = np.inf
    global_best_position = None
    
    while t < max_iter:
        # Update velocities and limit their magnitude
        V = update_velocity_gbest(V, S, P, chi, c1, c2)
        V = limit_velocity(V, X[0], X[1], alpha)

        # Update particles and ensure they are within bounds
        S = update_particles(V, S)
        S = check_bounds(S, X)

        # Update the best fitness and position
        new_fitness = evaluate(S, cost_function)
        # Update best positions
        P = update_best_positions(S, P, fitness, new_fitness)
        fitness = new_fitness
        # Increment time step
        t += num_particles
        
        # Check if current best is better than global best
        current_best_cost = np.min(fitness)
        if current_best_cost < global_best_cost:
            global_best_cost = current_best_cost
            global_best_position = P[np.argmin(fitness)]
            best_budget = t
        
        if t % 20000 == 0:
            S = initialization(X)
            V = initialization_velocity(S)
            P = S.copy()  
            
    # Report best position
    print(global_best_cost, (best_budget/200000)*100 )
    return global_best_position

In [13]:
# Define the search space
X = [0, 1]  
chi=0.729
c1 = 2.5
c2 = 1.5
# Define the number of particles and the dimension of the problem
num_particles = 200 #adjust the number of particles
dimension =57
alpha=0.01
max_iter=200000

In [14]:
# Initialize DataFrame for results with an index
results_df = pd.DataFrame(columns=['Result'])

# Repeat the genetic algorithm and append results to the DataFrame
for run_number in range(1, 26):  # Adjust the number of runs as needed
    result =  pso_gbest(cost_function, X, chi, c1, c2, alpha, max_iter)
    results_df.loc[run_number] = [' '.join(map(str, result))]

# Print the DataFrame with results
print(results_df)

# Save results to a file
results_df.to_csv('gbest200.txt', index=True)

448.44957946819886 40.0
448.44957946819886 10.0
448.44957946819886 69.6
448.4495794681988 39.4
448.4495794681988 69.89999999999999
448.4495794681988 49.1
448.4495794681988 8.5
448.4495794681988 19.900000000000002
448.44957946819875 69.8
448.4495794681988 19.400000000000002
448.44957946819886 9.700000000000001
448.4495794681988 19.6
448.44957946819886 8.4
448.44957946819886 29.4
448.4495794681988 20.0
448.4495794681988 99.5
448.4495794681988 69.69999999999999
448.4495794681988 19.5
448.4495794681988 8.7
448.4495794681988 79.7
448.4495794681988 20.0
448.44957946819886 8.9
448.44957946819875 99.5
448.4495794681988 19.8
448.4495794681988 88.9
                                               Result
1   0.2280237549171354 0.4477337352638675 0.438478...
2   0.25312949360099385 0.44773373661153826 0.4384...
3   0.2567944001829919 0.4477337351684017 0.438478...
4   0.30778378828321523 0.44773373493503993 0.4384...
5   0.2744016720597048 0.4477337361194027 0.438478...
6   0.284084378025655 0.44773

In [ ]:
#decision

In [8]:
def decision_function(x):
    M = decode_M(x[0])
    w = decode_coordinates(x)
    h = decode_hj(x, M)
    
    total_cost = 0
    # Define the coordinates of the lake triangle
    lake_coords = [(39.688367, 20.838671), (39.632539, 20.901448), (39.666708, 20.929109)]
          
    # Calculate total tij
    t = np.zeros((M, N))
    for i in range(M):
        for j in range(N):
            t[i][j] = haversine(w[i][0], w[i][1], customers[j][0], customers[j][1])
    
    # Calculate total cost Ci
    total_cost = 0
    for i in range(M):
        sdi = sum([demand[j] for j in range(N)])
        sti = sum([t[i][j] for j in range(N)])
        Ci = abs(Cop + sdi * C_pk(sdi)) + sti * Ckm
        
        
        if is_inside_triangle(w[i], lake_coords):
            # Add penalty to the total cost
            total_cost += 2000 + Ci
        else:
            total_cost += Ci
        
    return total_cost, M, w, h

In [21]:
# Load results from the file
results_df = pd.read_csv('lbest200.txt', index_col=0)

# Initialize an empty list to store detailed results
detailed_results = []

# Iterate through the results and evaluate the cost function
for run_number, result_str in results_df.iterrows():
    # Convert the result string to a NumPy array
    result = np.array(list(map(float, result_str['Result'].split())))

    # Evaluate the cost function for the result
    total_cost, M, w, h = decision_function(result)

    # Append results to the detailed list
    detailed_results.append({
        'Run': run_number,
        'M': M,
        'w': str(w).replace('[','').replace(']','').replace(')','').replace('(','').replace(',',''),  # Remove square brackets
        'h': str(h).replace('[','').replace(']','').replace(',','')  # Remove square brackets
    })

# Create a DataFrame from the detailed results list
detailed_results_df = pd.DataFrame(detailed_results)

# Save detailed results to a CSV file
detailed_results_df.to_csv('final_X/lbest200.txt', sep='\t', index=False)

# Print the detailed results DataFrame
print(detailed_results_df)

    Run  M                                      w  \
0     1  1    39.66764358454191 20.83998221218493   
1     2  1  39.667616016360306 20.839915798882174   
2     3  1  39.667644288234754 20.839946932932186   
3     4  1   39.66764367623201 20.839947428023574   
4     5  1   39.66764651910417 20.839945180551577   
5     6  1   39.66764411789864 20.839946384851345   
6     7  1    39.6676416929531 20.839946533607854   
7     8  1    39.66764365616076 20.83994787060689   
8     9  1   39.66764530893785 20.839946956645313   
9    10  1   39.66764376174498 20.839947550523018   
10   11  1   39.66764467910008 20.839947794335373   
11   12  1  39.667640908914095 20.839943869522887   
12   13  1     39.6676441497721 20.83994735714176   
13   14  1  39.667643961736445 20.839947215712936   
14   15  1   39.66763284702913 20.839949449358723   
15   16  1    39.66764399127467 20.83994696576508   
16   17  1  39.667643182626655 20.839949353039646   
17   18  1   39.66764436389765 20.839948315203